# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [75]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [76]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,5.45,84,96,16.59,TF,1686501265
1,1,iqaluit,63.7506,-68.5145,1.85,80,100,7.20,CA,1686501266
2,2,dudinka,69.4058,86.1778,10.90,50,71,7.30,RU,1686501266
3,3,grytviken,-54.2811,-36.5092,-1.53,87,97,5.39,GS,1686501266
4,4,yinzhu,35.8786,119.9753,19.79,72,87,1.54,CN,1686501266


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [77]:
%%capture --no-display

# Configure the map plot

map_plot_1= city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo= True,
    tiles= 'OSM',
    frame_width= 700,
    frame_height= 500,
    scale= 0.01,
    c= 'City',
    size= ('Humidity')*100)

# Display the map
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [78]:
# Narrow down cities that fit criteria and drop any results with null values
# Made my criteria to be min temp= 20C max temp= 35 0% cloudiness and humidity less than 75%
Vaca_no_cloud= city_data_df[city_data_df["Cloudiness"] == 0]
vaca_Min_temp= Vaca_no_cloud[Vaca_no_cloud["Max Temp"]>20]
vaca_max_temp= vaca_Min_temp[vaca_Min_temp["Max Temp"]<= 35]
ideal_weather= vaca_max_temp[vaca_max_temp["Humidity"]<= 75]


# Drop any rows with null values
ideal_weather= ideal_weather.dropna()

# Display sample data
ideal_weather.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,winslow,35.0242,-110.6974,24.10,15,0,3.60,US,1686501267
20,20,awjilah,29.1081,21.2869,34.67,23,0,4.33,LY,1686501268
28,28,cabo san lucas,22.8909,-109.9124,32.56,28,0,0.00,MX,1686501170
33,33,broome,42.2506,-75.8330,27.05,47,0,2.57,US,1686501270
40,40,jamestown,42.0970,-79.2353,25.85,46,0,3.60,US,1686501182


### Step 3: Create a new DataFrame called `hotel_df`.

In [79]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=ideal_weather.copy()
del hotel_df["Max Temp"]
del hotel_df["Cloudiness"]
del hotel_df["Wind Speed"]
del hotel_df["Date"]
del hotel_df["City_ID"]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= ""

# Display sample data
hotel_df.head()

,City,Lat,Lng,Humidity,Country,Hotel Name
10,winslow,35.0242,-110.6974,15,US,
20,awjilah,29.1081,21.2869,23,LY,
28,cabo san lucas,22.8909,-109.9124,28,MX,
33,broome,42.2506,-75.8330,47,US,
40,jamestown,42.0970,-79.2353,46,US,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [80]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation",
    "apiKey": geoapify_key,

}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame

    latitude= row['Lat']
    longitude= row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
winslow - nearest hotel: La Posada Hotel
awjilah - nearest hotel: استراحة اوجلا
cabo san lucas - nearest hotel: Comfort Rooms
broome - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
bethel - nearest hotel: Hampton Inn Danbury
tananger - nearest hotel: Hummeren hotell
vadso - nearest hotel: Scandic Vadsø
anar darah - nearest hotel: No hotel found
waat - nearest hotel: Baden-Powell Haus
farsan - nearest hotel: خانه معلم
san patricio - nearest hotel: No hotel found
dakhla - nearest hotel: Hotel El Massira
ilha de mocambique - nearest hotel: Escondidinho
stanley - nearest hotel: Hotel 52
arraial do cabo - nearest hotel: No hotel found
sandpoint - nearest hotel: Cedar Street Hotel & Suites
north battleford - nearest hotel: Beaver Hotel
crane - nearest hotel: No hotel found
rathdrum - nearest hotel: Grizzly Lodge
mozarlandia - nearest hotel: No hotel found
yarmouth - nearest hotel: The Inn At Cape Cod
ithaca - nearest hotel: Hilton Garden 

,City,Lat,Lng,Humidity,Country,Hotel Name
10,winslow,35.0242,-110.6974,15,US,La Posada Hotel
20,awjilah,29.1081,21.2869,23,LY,استراحة اوجلا
28,cabo san lucas,22.8909,-109.9124,28,MX,Comfort Rooms
33,broome,42.2506,-75.8330,47,US,No hotel found
40,jamestown,42.0970,-79.2353,46,US,DoubleTree Jamestown
47,bethel,41.3712,-73.4140,55,US,Hampton Inn Danbury
75,tananger,58.9333,5.6000,25,NO,Hummeren hotell
77,vadso,70.0744,29.7487,47,NO,Scandic Vadsø
87,anar darah,32.7587,61.6540,16,AF,No hotel found
153,waat,51.1454,6.4958,40,DE,Baden-Powell Haus


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [81]:
#%%capture --no-display

# Configure the map plot
map_plot_2= hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo= True,
    tiles= 'OSM',
    frame_width= 800,
    frame_height= 500,
    scale= 0.01,
    c= 'City',
    hover_cols = ["Hotel Name", "Country"]
    )

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)